# Splitting approach analysis {#sec-splitting_approach}

As previously noted, the main output of the TUG test (i.e., completioin time) can be decomposed into several components (i.e., subphases). In the literature, there seem to be different approaches to this decomposition: some authors break down the subphases of the test into *standing_up*, *first_walk*, *first_turn*, *second_walk*, *second_turn* and *sitting_down* [@zakaria2014;@madhushri2016;@beyea2017;@coelln2019]. Others consider the last two subphases into a single one combining the last turn and the sitting down activities into the *turn_to_sit* subphase [@salarian2010;@adame2012tug;@milosevic2013quantifying;@ansai2019performance]. 

From the two splitting approaches used in the literature, we train several models with four different datasets obtained from the dataset described in [@sec-dataset]:

- *sw_turning_and_sitting* (**sw_ts**): accelerometer and gyroscope data from a smartwatch, labelled with `SEATED`, `STANDING_UP`, `WALKING`, `TURNING` and `SITTING_DOWN` activities.
- *sp_turning_and_sitting* (**sp_ts**): accelerometer and gyroscope data from a smartphone, labelled with `SEATED`, `STANDING_UP`, `WALKING`, `TURNING` and `SITTING_DOWN` activities.
- *sw_turn_to_sit* (**sw_tts**): accelerometer and gyroscope data from a smartwatch, labelled with `SEATED`, `STANDING_UP`, `WALKING`, `TURNING` and `TURN_TO_SIT` activities.
- *sp_turn_to_sit* (**sp_tts**): accelerometer and gyroscope data from a smartphone, labelled with `SEATED`, `STANDING_UP`, `WALKING`, `TURNING` and `TURN_TO_SIT` activities.

The **ts** datasets were generated in the previous chapter, while the **tts** datasets were generated using the `01_relabel.py` script:

::: {add-from=libs/chapter4/pipeline/01_relabel.py code-fold='true' code-filename='01_relabel.py'}
```{.python}
```
:::

Then, for each dataset, $100$ models were trained using $80\%$ of the subjects as training subjects and the remaining $20\%$ as testing subjects. This proces was executed using the `02_splitting-evaluation.py` script.

::: {add-from=libs/chapter4/pipeline/02_splitting-evaluation.py code-fold='true' code-filename='02_splitting-evaluation.py'}
```{.python}
```
:::


## Results

In [1]:
from libs.chapter4.analysis.data_loading import load_reports
from libs.chapter4.analysis.statistical_tests import compare_splitting_approaches

reports = load_reports()

/opt/homebrew/Caskroom/miniforge/base/envs/tf310/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


[@tbl-comparison] compares the overall accuracy and F1-scores of `TURNING` and `SITTING_DOWN` from one side, and `TURN_TO_SIT` activities from the other side, obtained for each data source -- smartwatch (**sw**) or smartphone (**sp**) -- and splitting approach -- *turning* and *sitting down* (**ts**) or *turn_to_sit* (**tts**) -- from the trained models. The overall accuracy obtained with the models trained with the **ts** is statistically better than the ones trained with the **tts** datasets. Moreover, the F1-score of the `TURNING` activity is statistically worse in the **tts** datasets due to the reduced number of training samples for that activity compared with the **ts** datasets, caused by the fact that the `TURN_TO_SIT` activity includes the `TURNING` activity (which is one of the other activities to be individually detected). In addition, the F1-score of the `TURN_TO_SIT` activity is low compared with the scores of `TURNING` and `SITTING_DOWN` in the **ts** datasets.

In [2]:
#| label: tbl-comparison
#| tbl-cap: Overall accuracy and F1-scores of `TURNING`, `SITTING_DOWN` and `TURN_TO_SIT` for each data source and splitting approach.

def get_accuracy_and_f1_scores(reports):
    results = {}
    for dataset_key, dataset_reports in reports.items():
        source = dataset_key.split('_')[0]
        if source not in results:
            results[source] = {}

        dataset_accuracies = []
        dataset_turning = []
        dataset_specific = []

        specific_act, specific_score, key = ('SITTING_DOWN', 'f1-sitting-down', 'ts') if 'turning_and_sitting' in dataset_key else ('TURN_TO_SIT', 'f1-turn_to_sit', 'tts')
            
        for dataset_report in dataset_reports:
            dataset_accuracies.append(dataset_report['accuracy'])
            dataset_turning.append(dataset_report['TURNING']['f1-score'])
            dataset_specific.append(dataset_report[specific_act]['f1-score'])

        results[source][key] = {
            'accuracy': dataset_accuracies,
            'f1-turning': dataset_turning,
            f'{specific_score}': dataset_specific
        }

    return results

results = get_accuracy_and_f1_scores(reports)
comparison = compare_splitting_approaches(results, ['accuracy', 'f1-turning', 'f1-sitting-down', 'f1-turn_to_sit'])
comparison

/opt/homebrew/Caskroom/miniforge/base/envs/tf310/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: overflow encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/opt/homebrew/Caskroom/miniforge/base/envs/tf310/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: overflow encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)
/opt/homebrew/Caskroom/miniforge/base/envs/tf310/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: overflow encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/opt/homebrew/Caskroom/miniforge/base/envs/tf310/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: overflow encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)
/opt/homebrew/Caskroom/miniforge/base/envs/tf310/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: ove

,source,metric,turning_sitting,turn_to_sit,two-tailed test
0,sw,accuracy,0.848,0.809,"t(198)=13.459315696940894, p-val=0.0, power=1.0"
1,sw,f1-turning,0.795,0.565,"t(172.92193683041882)=44.34041443328367, p-val..."
2,sw,f1-sitting-down,0.804,-,-
3,sw,f1-turn_to_sit,-,0.735,-
4,sp,accuracy,0.857,0.789,"U=9036.0, p-val=0.0, power=1"
5,sp,f1-turning,0.846,0.529,"t(135.5274649940879)=57.06501597390656, p-val=..."
6,sp,f1-sitting-down,0.753,-,-
7,sp,f1-turn_to_sit,-,0.655,-


## Summary

From these results, we conclude that more accurate results are obtained when considering `TURNING` and `SITTING_DOWN` as separate activities, compared to combining them, since not only the overall accuracy of the prediction model is better, but also the predictability for the `TURNING` activity. Therefore, the first approach (separate activities) will be used in the implementation and evaluation of the system.

## Code reference

::: {.callout-tip}
The documentation of the Python functions employed in this section can be found in
 [Chapter 4 reference](reference/index.qmd#chapter-4):

- [`data_loading`](reference/chapter4.analysis.data_loading.qmd):
  - [`load_reports`](reference/chapter4.analysis.data_loading.qmd#libs.chapter4.analysis.data_loading.load_reports)
- [`statistical_tests`](reference/chapter4.analysis.statistical_tests.qmd):
  - [`compare_splitting_approaches`](reference/chapter4.analysis.statistical_tests.qmd#libs.chapter4.analysis.statistical_tests.compare_splitting_approaches)
:::